The goal of this notebook is to extend a previous implementation of the Logistic Regression classifier by implementing both $L1$ and $L2$ regularization.

In [1]:
import scipy.io
from scipy.special import expit as sigmoid
import numpy as np

The update rule for the Maximum Likelihood Estimation is given by 
$$w_j \leftarrow w_j + \epsilon x_j^i(y^i - g(\mathbf{w}^T\mathbf{x}^i))$$
Here, we loop over each data point $(y^i, \mathbf{x}^i)$ and update each weight $w_j$.
<br><br>We can choose to regularize our model to reduce overfitting. 
For example, $L1$ regularization seeks to minimize all learned weights. The update rule for $L1$ regularization is
$$w_j \leftarrow w_j + \epsilon \bigg[x_j^i(y^i - g(\mathbf{w}^T\mathbf{x}^i)) 
- \frac{\text{sign}(w_j)}{\lambda N} \bigg]$$
$L2$ regularization, on the other hand, seeks to minimize the number of non-zero weights. It's update rule is given by
$$w_j \leftarrow w_j + \epsilon \bigg[x_j^i(y^i - g(\mathbf{w}^T\mathbf{x}^i)) 
- \frac{w_j}{\lambda N} \bigg]$$

In [3]:
class LogisticRegression:
    
    def __init__(self):
        """
        Initialize a Logistic Regression Classifier object
        
        Params
        ---------
        weights - Learned weights for the Logistic Regression.
            Fit to the training data by calling the fit method.
            
        _epsilon - Hyperparameter for Logistic Regression controlling overfitting.
            
        _num_training - Number of training data points. Used for regularization.
        
        _lambda - Hyperparameter controlling regularization.
        """
        self.weights = None
        self._epsilon = None
        self._num_training = None
        self._lambda = None
        return None
    
    
    def accuracy(self, y, x):
        """
        Returns the classification accuracy.
        """
        prediction = self.predict(x)
        return np.mean(prediction == y)
    
    
    def fit(self, y, x, n=1, epsilon=.01, regularization=None, _lambda=1):
        """
        Learn the weights for a Logistic Regression Classifier from the data
        
        Params
        ---------
        y : Numpy array
            - The binary class labels for the data points
        x : Numpy array
            - A list of features for each data point
        n : int - Default: 1
            - The number of iterations for the training algorithm
        epsilon : float - Default: 0.01
            - Hyperparamter controlling overfitting of the model to the data
        regularization : {None, 'L1', 'L2'} - Default: None
            - Type of regularization to employ on the model
        _lambda : float
            - Hyper parameter for regularization
        """
        # Initialize the weight vector
        w_0 = np.zeros(x.shape[1])
        
        # Variables used for learning weights
        self._epsilon = epsilon
        self._num_training = x.shape[0]
        self._lambda = _lambda
        
        # Pick the correct update method
        if regularization == 'l1':
            #print 'L1 regularization'
            update_func = self._l1
        elif regularization == 'l2':
            #print 'L2 regularization'
            update_func = self._l2
        else:
            #print 'No regularization'
            update_func = self._no_reg
                    
        # Number of iterations
        for _ in range(n):

            # Loop over all the data points
            for i in range(x.shape[0]):
                
                y_minus_g = y[i] - sigmoid( np.dot(w_0, x[i]) )
                w_0 = update_func(y[i], x[i], y_minus_g, w_0)

        # Save the learned weights
        self.weights = w_0
        return None
    
    
    def _no_reg(self, y, x, y_minus_g, w_0):
        """
        Calculate the update to the weight vector with no regularization.
        
        Params
        ---------
        y : {0, 1}
            The binary class label for the data point
        x : Numpy array
            The feature vector for a single data point
        y_minus_g : float
            (y - sigmoid(w^T x))
        w_0 : float
            Previous weight vector
        epsilon : float
            Hyperparameter controlling overfitting
            
        Returns 
        ---------
            - New weight vector
        """
        # Initialize weight vector to return
        w_1 = np.zeros(len(w_0))
        
        ascent = self._epsilon * y_minus_g
        
        for j in range(len(x)):
            w_1[j] = w_0[j] + x[j]*ascent
            
        return w_1
    
    
    def _l1(self, y, x, y_minus_g, w_0):
        """
        Update rule for Logistic Regression with L1 regularization.
        
        Params
        ---------
        y : {0, 1}
            The binary class label for the data point
        x : Numpy array
            The feature vector for a single data point
        y_minus_g : float
            (y - sigmoid(w^T x))
        w_0 : float
            Previous weight vector
        epsilon : float
            Hyperparameter controlling overfitting
            
        Returns 
        ---------
            - New weight vector
        """
        # Initialize weight vector to return
        w_1 = np.zeros(len(w_0))
        
        for j in range(len(x)):
            reg = float(self._sign(w_0[j])) / (self._lambda*self._num_training)
            w_1[j] = w_0[j] + self._epsilon*(x[j]*y_minus_g - reg)
            
        return w_1
    
    
    def _l2(self, y, x, y_minus_g, w_0):
        """
        Update rule for Logistic Regression with L2 regularization.
        
        Params
        ---------
        y : {0, 1}
            The binary class label for the data point
        x : Numpy array
            The feature vector for a single data point
        y_minus_g : float
            (y - sigmoid(w^T x))
        w_0 : float
            Previous weight vector
        epsilon : float
            Hyperparameter controlling overfitting
            
        Returns 
        ---------
            - New weight vector
        """
        # Initialize weight vector to return
        w_1 = np.zeros(len(w_0))
        
        for j in range(len(x)):
            reg = float(w_0[j]) / (self._lambda*self._num_training)
            w_1[j] = w_0[j] + self._epsilon*(x[j]*y_minus_g - reg)
            
        return w_1
    
    
    def _sign(self, number): 
        """
        Returns the sign of a number (0 if number == 0)
        """
        return cmp(number,0)
    
    
    def predict(self, data):
        """
        Classifies each data point in x according to the weights learned from the fit method.

        Params
        ---------
        x : Numpy array
            The array of data points to classify.
            
        Returns
        ---------
            - Predicted class label for each data point.
        """
        prediction = []

        for x in data:
            prob_0 = self._sigmoidLikelihood(x, 0)
            prob_1 = self._sigmoidLikelihood(x, 1)

            if prob_0 > prob_1:
                prediction.append(0)
            else:
                prediction.append(1)

        return prediction
    
    
    def _sigmoidLikelihood(self, x, label):
        """
        Returns the sigmoid likelihood p(y=label|features; weights)
        
        x : Numpy array
            Feature set for a single data point
        """
        logit = sigmoid(np.dot(x, self.weights))
        
        if label == 0:
            return (1-logit)
        elif label == 1:
            return logit

In [4]:
from time import time

from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction import DictVectorizer as DV
#from sklearn.linear_model import LogisticRegression as sk_lr


from get_data import data
from logisticregression import LogisticRegression

In [5]:
# Load the data set
df = data()

X = df[ [col for col in df if col not in ['label', 'class']]]
y = df['class'].values

# Binarize the categorical data using a DictVectorizer
# This requires the data be fed in the form of Python dicts
vectorizer = DV(sparse=False)
X_binarized = vectorizer.fit_transform(X.to_dict(orient='records'))

X_binarized = np.array(X_binarized)

# Split into train, cv and test sets
X_train, X_test, y_train, y_test = train_test_split(X_binarized, y)

In [6]:
# My implementation
classifier = LogisticRegression()
classifier.fit(y_train, X_train, regularization='l1')
accuracy = 1 - classifier.accuracy(y_test, X_test)
print('Luigi Error: {}'.format((1-accuracy)))

Epsilon: 0.01
L1 regularization
Lambda: 1.0
Luigi Error: 0.782090652254


In [14]:
# sklearn
classifier = sk_lr()
classifier.fit(X_train, y_train)
accuracy = classifier.score(X_test, y_test)
print('Sklearn Error: {}'.format((1-accuracy)))

Sklearn Error: 0.202554968677


In [14]:
# Instantiate the model estimator
classifier = LogisticRegression()

# Fit the model to the training data
t0 = time()
classifier.fit(y_train, X_train)
t1 = time()

print 'Time to train classifier: {} seconds'.format(t1-t0)

# Apply the learned model on unseen data
prediction = classifier.predict(X_test)

accuracy = classifier.score(X_test, y_test)
error = (1 - accuracy)

print
print 'Test accuracy: {}'.format(accuracy)
print 'Test error: {}'.format(error)

print
print 'Train accuracy: {}'.format(classifier.score(X_train, y_train))

# View the learned coefficients and the intercept
#print(classifier.coef_)
#print(classifier.intercept_)

IndexError: tuple index out of range

In [3]:
data = scipy.io.loadmat('../hw2/hw2data.mat')
trainData = data['trainData']

y = trainData[:, 14]
x = trainData[:, :14]

In [4]:
classifier = LogisticRegression()
classifier.fit(y, x, n=500)
acc = classifier.accuracy(y, x)
print 'Accuracy: {}'.format(acc)

No regularization
Accuracy: 0.74


In [38]:
lambdas = np.linspace(1, 10, num=91)

In [39]:
for l in lambdas:
    classifier = LogisticRegression()
    classifier.fit(y, x, n=500, regularization='l1', _lambda=l)
    acc = classifier.accuracy(y, x)
    print 'Lambda: {} \tAccuracy: {}'.format(l, acc)

L1 regularization
Lambda: 1.0 	Accuracy: 0.66
L1 regularization
Lambda: 1.1 	Accuracy: 0.49
L1 regularization
Lambda: 1.2 	Accuracy: 0.63
L1 regularization
Lambda: 1.3 	Accuracy: 0.67
L1 regularization
Lambda: 1.4 	Accuracy: 0.6
L1 regularization
Lambda: 1.5 	Accuracy: 0.59
L1 regularization
Lambda: 1.6 	Accuracy: 0.69
L1 regularization
Lambda: 1.7 	Accuracy: 0.7
L1 regularization
Lambda: 1.8 	Accuracy: 0.67
L1 regularization
Lambda: 1.9 	Accuracy: 0.67
L1 regularization
Lambda: 2.0 	Accuracy: 0.74
L1 regularization
Lambda: 2.1 	Accuracy: 0.76
L1 regularization
Lambda: 2.2 	Accuracy: 0.48
L1 regularization
Lambda: 2.3 	Accuracy: 0.61
L1 regularization
Lambda: 2.4 	Accuracy: 0.8
L1 regularization
Lambda: 2.5 	Accuracy: 0.54
L1 regularization
Lambda: 2.6 	Accuracy: 0.76
L1 regularization
Lambda: 2.7 	Accuracy: 0.69
L1 regularization
Lambda: 2.8 	Accuracy: 0.69
L1 regularization
Lambda: 2.9 	Accuracy: 0.76
L1 regularization
Lambda: 3.0 	Accuracy: 0.67
L1 regularization
Lambda: 3.1 	Accura

In [40]:
for l in lambdas:
    classifier = LogisticRegression()
    classifier.fit(y, x, n=500, regularization='l2', _lambda=l)
    acc = classifier.accuracy(y, x)
    print 'Lambda: {} \tAccuracy: {}'.format(l, acc)

L2 regularization
Lambda: 1.0 	Accuracy: 0.75
L2 regularization
Lambda: 1.1 	Accuracy: 0.51
L2 regularization
Lambda: 1.2 	Accuracy: 0.61
L2 regularization
Lambda: 1.3 	Accuracy: 0.54
L2 regularization
Lambda: 1.4 	Accuracy: 0.26
L2 regularization
Lambda: 1.5 	Accuracy: 0.55
L2 regularization
Lambda: 1.6 	Accuracy: 0.27
L2 regularization
Lambda: 1.7 	Accuracy: 0.61
L2 regularization
Lambda: 1.8 	Accuracy: 0.55
L2 regularization
Lambda: 1.9 	Accuracy: 0.36
L2 regularization
Lambda: 2.0 	Accuracy: 0.78
L2 regularization
Lambda: 2.1 	Accuracy: 0.63
L2 regularization
Lambda: 2.2 	Accuracy: 0.59
L2 regularization
Lambda: 2.3 	Accuracy: 0.57
L2 regularization
Lambda: 2.4 	Accuracy: 0.27
L2 regularization
Lambda: 2.5 	Accuracy: 0.58
L2 regularization
Lambda: 2.6 	Accuracy: 0.6
L2 regularization
Lambda: 2.7 	Accuracy: 0.6
L2 regularization
Lambda: 2.8 	Accuracy: 0.59
L2 regularization
Lambda: 2.9 	Accuracy: 0.6
L2 regularization
Lambda: 3.0 	Accuracy: 0.6
L2 regularization
Lambda: 3.1 	Accurac